In [1]:
cd ../

c:\Users\Nicole\Desktop\MDS\capstone\canucks_mds_capstone


c:\Users\Nicole\miniconda3\envs\canucks_pricing\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import pandas as pd
from src.preprocessing import *

In [3]:
# CREATE A POPULARITY RANKING CSV 

# ranking from number of fans 
data = {
    'team': [
        "Chicago Blackhawks", "Boston Bruins", "Pittsburgh Penguins", "Detroit Red Wings",
        "Montreal Canadiens", "New York Rangers", "Toronto Maple Leafs", "Philadelphia Flyers",
        "Los Angeles Kings", "San Jose Sharks", "Washington Capitals",
        "Colorado Avalanche", "St. Louis Blues", "Minnesota Wild", "Tampa Bay Lightning",
        "Edmonton Oilers", "Buffalo Sabres", "Dallas Stars", "New Jersey Devils",
        "Nashville Predators", "Vegas Golden Knights", "Anaheim Ducks", "Winnipeg Jets",
        "Calgary Flames", "Ottawa Senators", "Columbus Blue Jackets", "New York Islanders",
        "Carolina Hurricanes", "Arizona Coyotes", "Florida Panthers", "Seattle Kraken"
    ],
    'ranking': list(range(1, 32))
}

df_fans = pd.DataFrame(data)


In [4]:
# ranking by social media 
data = {
    'team': [
        "Anaheim Ducks", "Arizona Coyotes", "Boston Bruins", "Buffalo Sabres",
        "Calgary Flames", "Carolina Hurricanes", "Chicago Blackhawks",
        "Colorado Avalanche", "Columbus Blue Jackets", "Dallas Stars",
        "Detroit Red Wings", "Edmonton Oilers", "Florida Panthers",
        "Los Angeles Kings", "Minnesota Wild", "Montreal Canadiens",
        "Nashville Predators", "New Jersey Devils", "New York Islanders",
        "New York Rangers", "Ottawa Senators", "Philadelphia Flyers",
        "Pittsburgh Penguins", "San Jose Sharks", "Seattle Kraken",
        "St. Louis Blues", "Tampa Bay Lightning", "Toronto Maple Leafs",
        "Vegas Golden Knights", "Washington Capitals", "Winnipeg Jets"
    ],
    'ranking': [
        26, 31, 5, 22, 25, 23, 1, 11, 29, 13, 6, 10, 32, 17, 16, 4, 27, 18,
        27, 6, 30, 11, 3, 14, 21, 19, 15, 2, 20, 9, 24
    ]
}
df_social = pd.DataFrame(data)

In [5]:
merged_df = pd.merge(df_social, df_fans, on='team', how='inner')
merged_df.loc[:, 'overall'] = merged_df['ranking_x'] + merged_df['ranking_y']

In [6]:
sorted_df  = merged_df.sort_values(by='overall')
sorted_df['overall_rank'] = sorted_df['overall'].rank(method='dense').astype(int)

In [7]:
first_quartile = sorted_df['overall_rank'].quantile(1/3)
second_quartile = sorted_df['overall_rank'].quantile(2/3)

# Assign ranks based on quartiles
sorted_df['opponent_rank'] = pd.cut(sorted_df['overall_rank'], bins=[-float('inf'), first_quartile, second_quartile, float('inf')], labels=[1, 2, 3], include_lowest=True)

In [8]:
# adjust some teams
sorted_df.loc[sorted_df['team'].isin(['Edmonton Oilers', 'Calgary Flames']), 'opponent_rank'] = 1
sorted_df.sort_values(by='opponent_rank', inplace=True)
popularity_rank = sorted_df[['team', 'opponent_rank']]

In [9]:
# add in abbreviated name to match name in data 
abbreviations = {
    'Chicago Blackhawks': 'Blackhawks',
    'Pittsburgh Penguins': 'Penguins',
    'Boston Bruins': 'Bruins',
    'Montreal Canadiens': 'Canadiens',
    'Toronto Maple Leafs': 'Leafs',
    'Detroit Red Wings': 'Wings',
    'New York Rangers': 'Rangers',
    'Philadelphia Flyers': 'Flyers',
    'Washington Capitals': 'Capitals',
    'Colorado Avalanche': 'Avalanche',
    'Calgary Flames': 'Flames',
    'Edmonton Oilers': 'Oilers',
    'Vegas Golden Knights': 'Vegas Golden Knights',
    'Buffalo Sabres': 'Sabres',
    'New Jersey Devils': 'Devils',
    'St. Louis Blues': 'Blues',
    'Dallas Stars': 'Stars',
    'Tampa Bay Lightning': 'Lightning',
    'Minnesota Wild': 'Wild',
    'Los Angeles Kings': 'Kings',
    'San Jose Sharks': 'Sharks',
    'Winnipeg Jets': 'Jets',
    'Nashville Predators': 'Predators',
    'Anaheim Ducks': 'Ducks',
    'Carolina Hurricanes': 'Hurricanes',
    'Seattle Kraken': 'Seattle Kraken',
    'New York Islanders': 'Islanders',
    'Columbus Blue Jackets': 'Blue Jackets',
    'Ottawa Senators': 'Senators',
    'Arizona Coyotes': 'Coyotes',
    'Florida Panthers': 'Panthers'
}


In [10]:
popularity_rank.loc[:, 'data_name'] = popularity_rank['team'].map(abbreviations)


C:\Users\Nicole\AppData\Local\Temp\ipykernel_22588\429505929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popularity_rank.loc[:, 'data_name'] = popularity_rank['team'].map(abbreviations)


In [11]:
popularity_rank = popularity_rank[['team', 'data_name', 'opponent_rank']]


In [12]:
popularity_rank.to_csv('data/opponent_popularity.csv', index=False)

In [13]:
# INCORPORATE POPULARITY COLUMN INTO DATAFRAME 

In [14]:
df = pd.read_parquet('data/short_summary/combined.parquet')


In [15]:
df_ranking = pd.read_csv('data/opponent_popularity.csv')


,team,data_name,opponent_rank
0,Chicago Blackhawks,Blackhawks,1
1,Pittsburgh Penguins,Penguins,1
2,Boston Bruins,Bruins,1
3,Montreal Canadiens,Canadiens,1
4,Toronto Maple Leafs,Leafs,1
5,Detroit Red Wings,Wings,1
6,New York Rangers,Rangers,1
7,Philadelphia Flyers,Flyers,1
8,Washington Capitals,Capitals,1
9,Colorado Avalanche,Avalanche,1


In [16]:
df = extract_opponent_from_event(df)


In [17]:
# function to add in team ranking
def opponent_popularity(df):
    """adds in popularity ranking for the oppponent team (use after `extract_opponent_from_event`), 
    returns df with 'opponent_rank' column"""

    df_ranking = pd.read_csv('data/opponent_popularity.csv')
    merged_df = pd.merge(df, df_ranking[['data_name', 'opponent_rank']], left_on='opponent', right_on='data_name', how='left')
    merged_df['opponent_rank'] = merged_df['opponent_rank'].fillna(-1).astype(int)
    return(merged_df)

In [18]:
df_rank = opponent_popularity(df)

In [21]:
df_rank.tail()

,identifier,event_id,event_name,venue,calculate_date,price_level_name,cap,revenue_to_date,s/t-rate,opens,...,archtics_event_name,dl_rowhash,dl_iscurrent,dl_isdeleted,dl_recordstartdateutc,dl_recordenddateutc,dl_load_id,opponent,data_name,opponent_rank
748891,AllData_11005D05E8F553A7_Rogers Arena_Canucks_...,11005D05E8F553A7,Canucks vs. Seattle Kraken,Rogers Arena,08/16/2022,28: 28/R,757,0.0,0.0,0,...,None,d0b754844b56cf34220c04e3901aef2d40a0870d166e6e...,True,False,2024-04-11T09:59:14.544526-07:00,NaN,0,Seattle Kraken,Seattle Kraken,3
748892,AllData_11005D05E8F553A7_Rogers Arena_Canucks_...,11005D05E8F553A7,Canucks vs. Seattle Kraken,Rogers Arena,08/16/2022,29: 29/S,176,0.0,0.0,0,...,None,1afa65a68b4e72e3c4a1ea4f6c53c6dc7af47beb4f77f8...,True,False,2024-04-11T09:59:14.544526-07:00,NaN,0,Seattle Kraken,Seattle Kraken,3
748893,AllData_11005D05E8F553A7_Rogers Arena_Canucks_...,11005D05E8F553A7,Canucks vs. Seattle Kraken,Rogers Arena,08/16/2022,30: 30/T,76,0.0,0.0,0,...,None,8a4088266ede8afe31f1a585a40ed3e17c7692cdc54b59...,True,False,2024-04-11T09:59:14.544526-07:00,NaN,0,Seattle Kraken,Seattle Kraken,3
748894,AllData_11005D05E8F553A7_Rogers Arena_Canucks_...,11005D05E8F553A7,Canucks vs. Seattle Kraken,Rogers Arena,08/16/2022,31: 31/U,28,0.0,0.0,0,...,None,920ea28be7a904070c6233ac3a72e63a51cc87869f8f74...,True,False,2024-04-11T09:59:14.544526-07:00,NaN,0,Seattle Kraken,Seattle Kraken,3
748895,AllData_11005D05E8F553A7_Rogers Arena_Canucks_...,11005D05E8F553A7,Canucks vs. Seattle Kraken,Rogers Arena,08/16/2022,32: 32/V,80,0.0,0.0,0,...,None,3d420846d50e96840ef9a69a56212a4bd67fadffafc628...,True,False,2024-04-11T09:59:14.544526-07:00,NaN,0,Seattle Kraken,Seattle Kraken,3


In [19]:
# CHECK

# ranking 1 
opponents_rank_1 = df_rank.loc[df_rank['opponent_rank'] == 1, 'opponent'].unique()
print("Rank 1:")
print(opponents_rank_1)

# rank 2 
opponents_rank_2 = df_rank.loc[df_rank['opponent_rank'] == 2, 'opponent'].unique()
print("\nRank 2:")
print(opponents_rank_2)

# Unique opponent names for rank value of 3
opponents_rank_3 = df_rank.loc[df_rank['opponent_rank'] == 3, 'opponent'].unique()
print("\nRank 3:")
print(opponents_rank_3)

# Unique opponent names for rank value of -1 (NaN)
opponents_rank_na = df_rank.loc[df_rank['opponent_rank'] == -1, 'opponent'].unique()
print("\nRank -1 (NaN):")
print(opponents_rank_na)

Rank 1:
['Flames' 'Blackhawks' 'Leafs' 'Capitals' 'Wings' 'Canadiens' 'Avalanche'
 'Flyers' 'Penguins' 'Bruins' 'Rangers' 'Oilers']

Rank 2:
['Vegas Golden Knights' 'Kings' 'Stars' 'Sharks' 'Wild' 'Sabres'
 'Lightning' 'Blues' 'Devils']

Rank 3:
['Seattle Kraken' 'Coyotes' 'Senators' 'Jets' 'Ducks' 'Islanders'
 'Blue Jackets' 'Predators' 'Hurricanes' 'Panthers']

Rank -1 (NaN):
['TBD at Vancouver Canucks Round 1 Home Game 3'
 'TBD at Vancouver Canucks Round 1 Home Game 4'
 'TBD at Vancouver Canucks Round 1 Home Game 2'
 'Test Event - DO NOT PURCHASE'
 'TBD at Vancouver Canucks Round 1 Home Game 1'
 'Canucks v Islanders capacity updates in progress!'
 'Canucks v Coyotes capacity updates in progress!'
 'Cancelled ticket due to 50% capacity Canucks vs Maple Leafs' 'TBD'
 'TBD at Vancouver Canucks Round 2 Home Game 3'
 'TBD at Vancouver Canucks Round 2 Home Game 4'
 'TBD at Vancouver Canucks Round 2 Home Game 2'
 'TBD at Vancouver Canucks Round 2 Home Game 1'
 'TBD at Vancouver Canucks Rou